In [ ]:
import glob
import json

## load the labels

In [ ]:
jsonfiles = glob.glob("./labels/*.json")

In [ ]:
all_annotations = []
for jf in jsonfiles:
    label = json.load(open(jf))
    for labels in label:
        if labels["Label"] != "Skip":
            all_annotations.append(labels)

In [ ]:
print(len(all_annotations))

## all paths

In [ ]:
import os

In [ ]:
paths = glob.glob("/root/data/aquabyte-images/erko-rotoy/*/1/*")
path_dic = {}
for p in paths:
    path_dic[os.path.basename(p)] = p

# create new dataset

In [ ]:
import numpy as np

In [ ]:
freckles_head =[]
for annotation in all_annotations:
    parts = annotation["Label"]
    if "Head" in parts:
        for head in parts["Head"]:
            # print(head)
            tmp = {"local_path": path_dic[annotation["External ID"]]}
            geometry = head["geometry"]
            xy = np.array([[k["x"], k["y"]] for k in geometry])
            xmin, xmax = np.min(xy[:, 0]), np.max(xy[:, 0])
            ymin, ymax = np.min(xy[:, 1]), np.max(xy[:, 1])
            tmp["bbox"] = [int(xmin), int(ymin), int(xmax), int(ymax)]
            tmp["freckles"] = []
            if "Freckles" in parts:
                for freckle in parts["Freckles"]:
                    x = freckle["geometry"]["x"]
                    y = freckle["geometry"]["y"]
                    if xmin < x < xmax and ymin < y < ymax:
                        tmp["freckles"].append([x, y])
            if len(tmp["freckles"]) > 0:
                freckles_head.append(tmp)
            # tmp["head_bbox"]
            
        

In [ ]:
with open("./labels/all.json", "w") as f:
    json.dump(freckles_head, f)

In [ ]:
print(len(freckles_head))

## plot random example

In [ ]:
import matplotlib.pyplot as plt
import skimage.io as io

In [ ]:
rfreckle = np.random.choice(freckles_head)

In [ ]:
image = io.imread(rfreckle["local_path"])
x1, y1, x2, y2 = rfreckle["bbox"]

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
ax.imshow(image[y1:y2, x1:x2])
for freckle in rfreckle["freckles"]:
    plt.scatter(freckle[0]-x1, freckle[1]-y1, c='r', marker="o", s=50)
plt.show()

In [ ]:
x1, y1